In [18]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')

df["대여일자"] = pd.to_datetime(df['대여일자'])
df["day_name"] = df.대여일자.dt.day_of_week
df["day_name"] = df.day_name.replace(0,"Monday").replace(1,"Tuesday").replace(2,"Wednesday")\
                            .replace(3,"Thursday").replace(4,"Friday").replace(5,"Saturday").replace(6,"Sunday")
stack = df.groupby(["day_name"])["day_name"].count()
#stack = df.day_name.value_counts().to_frame()
print(stack)

day_name
Friday       108877
Monday       107568
Saturday     101299
Sunday        90918
Thursday      55977
Tuesday      103704
Wednesday    110607
Name: day_name, dtype: int64


In [ ]:
# 집계 함수 sum, count , size
# sum() 컬럼내의 값들의 합
# count() 컬럼의 row를 센다
# groupby("col")[non] 일때 size 를 쓰면 유용하다
stack = df.groupby(["day_name","대여소번호"])["이용건수"].count().reset_index(drop = False)
stack = stack.sort_values(by = ["day_name","이용건수"],ascending=False)
stack = stack.drop_duplicates("day_name", keep = "first").reset_index(drop = True)
display(stack)
result = df.groupby(['day_name','대여소번호']).size().to_frame('size').sort_values(['day_name','size'],ascending=False).reset_index()
answer  = result.drop_duplicates('day_name',keep='first').reset_index(drop=True)
display(answer)

,day_name,대여소번호,이용건수
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


,day_name,대여소번호,size
0,Wednesday,502,282
1,Tuesday,502,267
2,Thursday,2715,137
3,Sunday,502,372
4,Saturday,502,378
5,Monday,502,242
6,Friday,502,277


나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 

가장 높은 비율을 가지는 나이대를 확인하라. 

일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

In [ ]:
# 1)
daily = df[df.대여구분코드.isin(['일일권','일일권(비회원)'])].연령대코드.value_counts().sort_index()
total = df.연령대코드.value_counts().sort_index()

ratio = (daily /total).sort_values(ascending=False)
print(ratio)
print('max ratio age ',ratio.index[0])

# 2)
stack = df.groupby(["연령대코드","대여구분코드"]).size().to_frame("size").reset_index()
stack = stack.groupby(["연령대코드"])["size"].sum().reset_index()

stack2 = df.groupby(["연령대코드","대여구분코드"]).size().to_frame("sum").reset_index()
stack2 = stack2.loc[(stack2.대여구분코드 == "일일권")|(stack2.대여구분코드 == "일일권(비회원)")]
stack2 = stack2.groupby(["연령대코드"])["sum"].sum().reset_index()
result = pd.concat([stack,stack2],axis = 1)
result["ratio"] = result["sum"]/result["size"]
print(result[["연령대코드","ratio"]])

  연령대코드 연령대코드     ratio
0   20대   20대  0.239913
1   30대   30대  0.166702
2   40대   40대  0.110585
3   50대   50대  0.065414
4   60대   60대  0.042462
5  70대~  70대~  0.048962
6  ~10대  ~10대  0.312525


In [ ]:
stack = df.loc[df["연령대코드"] == "20대"]
mean_ = stack["이동거리"].mean()
stack = stack.loc[stack["이동거리"] >= mean_]
stack = stack.sort_values(by = ["대여일자","대여소번호"], ascending = False).reset_index(drop=True)
result = pd.DataFrame(stack.loc[0:199, "탄소량"])
result["탄소량"] = result["탄소량"].astype(float)
result = result["탄소량"].mean()
print(round(result, 3))

1.613


In [146]:
cod = (df["대여일자"] == "2021-06-07")&(df["연령대코드"] == "~10대")
df.loc[cod]["이용건수"].median()

1.0

In [ ]:
# groupby 2번
stack = df.loc[((df["대여시간"] == 6)|(df["대여시간"] == 7)|(df["대여시간"] == 8))&(df["day_name"].isin(['Tuesday', 'Wednesday', 'Thursday', 'Friday','Monday']))]
stack = stack.groupby(["대여시간","대여소번호"])["이용건수"].size().to_frame("이용횟수")
stack = stack.sort_values(by = ["대여시간","이용횟수"], ascending = False).groupby(["대여시간"]).head(3)

이용횟수
대여시간 대여소번호      
8    2701    119
     646     115
     1152     92
7    259     104
     230      77
     726      77
6    2744     45
     1125     40
     1028     36

In [168]:
mean_ = df["이동거리"].mean()
stack = df.loc[df["이동거리"] >= mean_]
result = stack["이동거리"].std()
print(result)

5092.139707505356


In [175]:
male = df.loc[df.성별.isin(["M","m"])]
female = df.loc[df.성별.isin(["F",'f'])]
mean1 = male.이동거리.mean()
print(mean1)

3209.1108714530546
